In [33]:
import datetime
import numpy as np
import pandas as pd

In [2]:
%run ../tasking_manager_stats/data_management

Data file found and the project has not been changed. It won't be downloaded again.


In [22]:
start_date = datetime.datetime.strptime('2020-11-23 18:00:00', '%Y-%m-%d %H:%M:%S')
start_date -= datetime.timedelta(hours=1) # Winter
timestamps = pd.date_range(start_date, start_date + datetime.timedelta(hours=2), freq='5min')
timestamps

DatetimeIndex(['2020-11-23 17:00:00', '2020-11-23 17:05:00',
               '2020-11-23 17:10:00', '2020-11-23 17:15:00',
               '2020-11-23 17:20:00', '2020-11-23 17:25:00',
               '2020-11-23 17:30:00', '2020-11-23 17:35:00',
               '2020-11-23 17:40:00', '2020-11-23 17:45:00',
               '2020-11-23 17:50:00', '2020-11-23 17:55:00',
               '2020-11-23 18:00:00', '2020-11-23 18:05:00',
               '2020-11-23 18:10:00', '2020-11-23 18:15:00',
               '2020-11-23 18:20:00', '2020-11-23 18:25:00',
               '2020-11-23 18:30:00', '2020-11-23 18:35:00',
               '2020-11-23 18:40:00', '2020-11-23 18:45:00',
               '2020-11-23 18:50:00', '2020-11-23 18:55:00',
               '2020-11-23 19:00:00'],
              dtype='datetime64[ns]', freq='5T')

In [3]:
project_id = 9849
db = Database(project_id)

Data file not found locally and on GitHub, it will be downloaded again.
Download tasking manager data for project 9849


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 87/87 [03:26<00:00,  4.93s/it]


In [7]:
for task_id in db.get_task_ids():
    if str(task_id) not in db.get_task_history():
        print(f'Task {task_id} missing')
        continue
    task_data = db.get_task_history()[str(task_id)]
    

In [10]:
task_id = db.get_task_ids()[0]
task_data = db.get_task_history()[str(task_id)]

In [12]:
task_data['taskHistory']

[{'action': 'LOCKED_FOR_MAPPING',
  'actionBy': 'MM14',
  'actionDate': '2020-11-28T12:01:51.236675Z',
  'actionText': '00:03:16.927169',
  'historyId': 13735017,
  'issues': None,
  'pictureUrl': None,
  'taskId': None},
 {'action': 'LOCKED_FOR_MAPPING',
  'actionBy': 'heidilam',
  'actionDate': '2020-11-27T04:19:22.501003Z',
  'actionText': '00:00:11.475808',
  'historyId': 13718505,
  'issues': None,
  'pictureUrl': None,
  'taskId': None},
 {'action': 'LOCKED_FOR_MAPPING',
  'actionBy': 'heidilam',
  'actionDate': '2020-11-27T03:52:18.537029Z',
  'actionText': '00:26:57.162846',
  'historyId': 13718389,
  'issues': None,
  'pictureUrl': None,
  'taskId': None},
 {'action': 'LOCKED_FOR_MAPPING',
  'actionBy': 'Mhenking',
  'actionDate': '2020-11-24T16:41:36.674341Z',
  'actionText': '00:00:55.968501',
  'historyId': 13674535,
  'issues': None,
  'pictureUrl': None,
  'taskId': None},
 {'action': 'LOCKED_FOR_MAPPING',
  'actionBy': 'micmaz',
  'actionDate': '2020-11-24T16:40:41.53417

In [41]:
task_hist = task_data['taskHistory'][-8]
date = pd.to_datetime(task_hist['actionDate'])
# task_states = np.zeros(len(timestamps))
# task_states[date < timestamps]
date + pd.to_timedelta(task_hist['actionText'])

Timestamp('2020-11-23 17:50:48.860275')

In [60]:
def compute_task_states(task_data, timestamps):
    task_states = np.zeros(len(timestamps))
    task_locked = np.zeros(len(timestamps))
    for task_hist in reversed(task_data['taskHistory']):
        date = pd.to_datetime(task_hist['actionDate'])
        timestamp = (date - timestamps[0]).days
        if task_hist['action'].startswith('LOCK'):
            task_locked[date < timestamps] = 1
            task_locked[date + pd.to_timedelta(task_hist['actionText']) < timestamps] = 0
            continue
        if task_hist['action'] != 'STATE_CHANGE':
            continue
        if task_hist['actionText'] == 'MAPPED':
            task_states[date < timestamps] = 1
            continue
        if task_hist['actionText'] == 'INVALIDATED':
            task_states[date < timestamps] = 2
            continue
        if task_hist['actionText'] == 'VALIDATED':
            task_states[date < timestamps] = 3
            continue
        if task_hist['actionText'] == 'BADIMAGERY':
            task_states[date < timestamps] = 4
            continue
    return task_states, task_locked

In [65]:
def get_task_states_and_locked_tasks(db, timestamps):
    tasks_states = dict()
    tasks_locked = dict()
    for task_id in db.get_task_ids():
        task_states, task_locked = compute_task_states(db.get_task_history()[str(task_id)], timestamps)
        tasks_states[task_id] = task_states
        tasks_locked[task_id] = task_locked
    return tasks_states, tasks_locked

In [66]:
tasks_states, tasks_locked = get_task_states_and_locked_tasks(db, timestamps)

In [68]:
tasks_locked[task_id]

array([ 0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])